In [1]:
#Boundary Flow Constraint movie
import numpy as np
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import time
import math
import scipy.io as sio

sns.set_style("whitegrid", {'axes.grid':False})

plt.clf()
plt.close()
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Setting parameters

foreground_speed = 2
background_speed = 1
plot_width = 20
plot_height = 10
num_lines = int(plot_height/2)
dot_size = 6
num_frames = 3
motion_range = plot_width/15
num_dots = 100
line_thickness = 3
line_color = "black"
dot_color = "black"
mat_file_name = "images.mat"

In [3]:
def fig2rgb_array(fig, ax):
    can = fig.canvas
    can.draw()
    can.blit(ax.bbox)
    buf = can.tostring_rgb()
    ncols, nrows = can.get_width_height()
    return np.fromstring(buf, dtype=np.uint8).reshape(nrows, ncols, 3)

In [30]:
# Add in the null space in the middle
plot_middle = plot_width/2
nullBoundary = (plot_middle-2*motion_range, plot_middle+2*motion_range)

#Generate random dots
left_dots_x = nullBoundary[0]*np.random.rand(num_dots//2)
left_dots_y = plot_height*np.random.rand(num_dots//2)
right_dots_x = nullBoundary[0]*np.random.rand(num_dots//2) + nullBoundary[1]
right_dots_y = plot_height*np.random.rand(num_dots//2)

#for x, y in dots_x, dots_y:
fig = plt.figure()

#Main updating plot
x_inc = 0.5
left_motion = True

#Calculate number of cycles the movie has
num_cycles = int(x_inc*num_frames/(2*math.pi))

In [36]:

matrix = []
for frame in range(num_frames):
    x = x_inc*frame
    cycle = int(x/(2*math.pi))
    #Change coherence once half the number of cycles are completed
    if cycle > num_cycles//2:
        left_motion = False
    shift = math.sin(x)*motion_range
    if left_motion:
        lengthLine = shift + 0.5*plot_width
    else:
        lengthLine = 0.5*plot_width - shift
        
    ax = fig.add_subplot(111)
    ax.set_xlim(0, plot_width)
    ax.set_ylim(0, plot_height)
    
    #Plot all new horizontal lines for this frame
    for i in range(num_lines):
        update_left_line = Line2D([0, lengthLine], [2*i+1, 2*i+1], lw=line_thickness, c=line_color)
        update_right_line = Line2D([lengthLine, plot_width], [2*i, 2*i], lw=line_thickness, c=line_color)
        ax.add_line(update_left_line)
        ax.add_line(update_right_line)

    #Plot vertical line and dots
    ax.add_line(Line2D([lengthLine, lengthLine], [0, plot_height], lw=line_thickness, c=line_color))
    ax.scatter(left_dots_x+shift, left_dots_y, dot_size, c=dot_color)
    ax.scatter(right_dots_x-shift, right_dots_y, dot_size, c=dot_color)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    fig.tight_layout()
    matrix.append(fig2rgb_array(fig, ax))
    fig.clf()

sio.savemat(mat_file_name, matrix)    
